### SELECT FIRST
select kernal env 
    EXAMPLE : env(Python 3.12.2)

###### NOTE : PLEASE DON'T SELECT GLOBAL ENV


In [4]:
import pandas as pd
import os
import numpy as np
import re

In [5]:
df = pd.read_csv('data_step_4_scrap_merged/scrap_df.csv')

In [6]:
df = df.rename(columns={
                "Document Number":"doc_number",
                "Status":"status",
                "Case Number":"case_number",
                "Name of Court":"court_name",
                "File Date":"file_date",
                "Date of Entry":"entry_date",
                "Expiration Date":"expiry_date",
                "Amount Due":"amount_due",
                "Interest Rate":"interest_rate",
                "creditors":"creditors",
                "debtors":"debtors",
                "Original Document":"original_doc",
                "Amount Remaining":"amount_remaining",
                "url":"url"
                })

In [7]:

df['entry_date_format'] = df['entry_date']

In [8]:
df['Year'] = df['entry_date_format'].str.split('/').str[-1]
df['Month'] = df['entry_date_format'].str.split('/').str[0]
df['Day'] = df['entry_date_format'].str.split('/').str[1]

In [9]:
df = df[(df['Year']>='2001') & (df['Month']>='01') & (df['Day']>='01')]

In [10]:
df.drop(columns=['Year','Month','Day'], inplace=True)

In [11]:
df['creditor_first_line'] = df['creditors'].str.split('\n').str[0]
df['creditor_last_line'] = df['creditors'].str.split('\n').str[-1]
df['creditor_address'] = df['creditors'].str.replace(',','').str.split('\n').str[1:-1].apply(lambda x: str(x).replace('[','').replace(']','').replace('\'', ''))
df['is_creditor_company'] = df['creditor_first_line'].str.contains("INC|HOUR|DIV|ENTERPRISE|ACADEMY|0|1|2|3|4|5|6|7|8|9|:|5TH|RENT|OWNRS|BIG|MIAMI|SHOE|REPEAT|EVOLUTION|FLORIDA|ADVERTISING|PLACE|SCHOOL|MOBILITY|LAND|STATE|SUPER|VENTURES|SECURITY|CYBER|NATIONAL|COLLECTORS|RESOLUTION|COTTAGE|COUNTRY|GAINESVILLE|REVENUE|PROPERTIES|INDUSTRIAL|COPORATION|INTEREST|CITY|COMMUNICATIONS|COMPAN|COUNTY|DIVISION|HOSPITAL|L.L.C.|SUPPLY|GROUP|TECHNOLOGIES|ASSOCIATES|LIMITED|INSURANCE|L.P|JOINT|DESIGN|SRVCS|BANK|NORTH|INVEST|LTD|CENTER|TRUST|POPULAR|CLINIC|CREDIT|COMPANY|TELEPHONE|D.M.D.|UNIT|LLC|INDUSTRIES|C/O|CORPORATION|FINANCIAL|CREDIT|BANK|EMPLOYERS|FEDERAL|JEWELRY|AVENUE|ELECTRICAL|DISTRUBUTORS|UNIVERSITY|INNOVATIONS|SERVICE|SERVICES|CORP|CREATION|CERTIFIED|REPAIR|AUTO|TRANPORTATION|&|CO.|STAFFING|FIRM|TRADE|ENTERTAINMENT|TECHNOLOGY|FINANCE|CLEANING|SOLUTIONS|CASINO|HOLDINGS|HEALTH|SYSTEMS|L L C|FOODS|OIL|CLUB|UNIFUND|OFFICE| CO |CHANGE|PROJECT|ZOOLOGICAL|SPORTS|SHOP|BODY|BOATS|ABOUT|WIRELESS|WIRE|BEGINNING")

In [12]:
df['creditor_company_name'] = np.where(df['is_creditor_company']==True,df['creditor_first_line'],'')
df['creditor_name'] = np.where(df['is_creditor_company']==False,df['creditor_first_line'].str.split(',').str[0],'')
df['creditor_city'] = np.where(
                            df['creditor_last_line'].str.contains(','),
                            df['creditor_last_line'].str.split(',').str[0],
                            np.where(
                                df['creditor_last_line'].str.split(' ').str.len() > 2,
                                df['creditor_last_line'].str.split(' ').str[0:-2].apply(lambda x: str(x).replace('[','').replace(']','').replace('\'', '')).str.replace(',',' ').str.strip(),
                                df['creditor_last_line'].str.split(' ').str[0]
                                )
                            )


df['creditor_zip'] = np.where(
                            df['creditor_last_line'].str.contains(','),
                           
                            df['creditor_last_line'].str.split(',').str[1].str.replace('.',' ').str.split(' ').str[-1].str.extract(pat='(\d+)', expand=False).str[0:6],
                            np.where(
                                df['creditor_last_line'].str.split(' ').str.len() > 2,
                                df['creditor_last_line'].str.split(' ').str[-1].str.extract(pat='(\d+)', expand=False).str[0:6],
                                ''
                                )
                            )

df['creditor_state'] = np.where(
                            df['creditor_last_line'].str.contains(','),
                            df['creditor_last_line'].str.split(',').str[-1].str.replace('.',' ').str.strip().str.split(' ').str[0],
                            np.where(
                                df['creditor_last_line'].str.split(' ').str.len() > 2,
                                df['creditor_last_line'].str.split(' ').str[-2],
                                df['creditor_last_line'].str.split(' ').str[-1].str.extract(pat='([a-z]+)', expand=False).str[0:2]
                                
                                )
                            )

<>:17: SyntaxWarning: invalid escape sequence '\d'
<>:20: SyntaxWarning: invalid escape sequence '\d'
<>:17: SyntaxWarning: invalid escape sequence '\d'
<>:20: SyntaxWarning: invalid escape sequence '\d'
C:\Users\PIPLUSPI\AppData\Local\Temp\ipykernel_2444\2026663877.py:17: SyntaxWarning: invalid escape sequence '\d'
  df['creditor_last_line'].str.split(',').str[1].str.replace('.',' ').str.split(' ').str[-1].str.extract(pat='(\d+)', expand=False).str[0:6],
C:\Users\PIPLUSPI\AppData\Local\Temp\ipykernel_2444\2026663877.py:20: SyntaxWarning: invalid escape sequence '\d'
  df['creditor_last_line'].str.split(' ').str[-1].str.extract(pat='(\d+)', expand=False).str[0:6],


In [13]:
df['creditor_fname'] = df['creditor_name'].str.split(' ').str[0]
df['creditor_lname'] = df['creditor_name'].str.split(' ').str[-1]
df['creditor_mname'] = np.where(df['creditor_name'].str.len()>2,
                               df['creditor_name'].str.split(' ').str[1:-1].apply(lambda x: str(x).replace('[','').replace(']','').replace('\'', '')).str.replace(',',' ').str.strip()
                               ,'')
df['creditor_full_name'] = df['creditor_name'] 


In [14]:
df['debtors'] = df['debtors'].str.split('\n \n')
df = df.explode('debtors')

In [15]:
df['debtors_arrays'] = df['debtors'].str.replace(',','|').str.split('\n')
df['debtors_arrays_count'] = df['debtors_arrays'].str.len()

df['debtors_line_number_FEI_EIN'] = np.where(
    df['debtors_arrays'].str[-1].str.contains('FEI/EIN'),
    df['debtors_arrays_count'],
    0
)
df['debtors_line_number_document'] = np.where(
    df['debtors_arrays'].str[-1].str.contains('Document Number'),
    df['debtors_arrays_count'],
    np.where(
        df['debtors_arrays'].str[-2].str.contains('Document Number'),
        df['debtors_arrays_count']-1,
        0
    )
)

df['debtors_full_name'] = df['debtors'].str.split('\n').str[0]

df['debtors_address_last_line_number'] = np.where(
                        df['debtors_line_number_document']>0,
                        df['debtors_line_number_document']-1,
                        np.where(
                            df['debtors_line_number_FEI_EIN']>0,
                                df['debtors_line_number_FEI_EIN']-1,
                                df['debtors_arrays_count']
                            )
                    ).astype(np.int32)

def getDebAddress(last_line,deb_lines):
    return deb_lines[1:last_line]
df['debtors_address_array'] = df.apply(lambda x:getDebAddress(x['debtors_address_last_line_number'],x['debtors_arrays']),axis=1)

In [16]:
def getDebDocument(address_line,deb_lines):
    if address_line > 0:
        return deb_lines[address_line-1]
    else:
        return ''

df['debtors_document_number'] = df.apply(lambda x:getDebDocument(x['debtors_line_number_document'],x['debtors_arrays']),axis=1)


In [17]:
def getDebFEI_EIN(address_line,deb_lines):
    if address_line > 0:
        return deb_lines[address_line-1]
    else:
        return ''

df['debtors_FEI_EIN_number'] = df.apply(lambda x:getDebFEI_EIN(x['debtors_line_number_FEI_EIN'],x['debtors_arrays']),axis=1)

In [18]:
df['is_debtor_company'] = df['debtors_full_name'].str.contains("INC|HOUR|DIV|ENTERPRISE|ACADEMY|0|1|2|3|4|5|6|7|8|9|:|5TH|RENT|OWNRS|BIG|MIAMI|SHOE|REPEAT|EVOLUTION|FLORIDA|ADVERTISING|PLACE|SCHOOL|MOBILITY|LAND|STATE|SUPER|VENTURES|SECURITY|CYBER|NATIONAL|COLLECTORS|RESOLUTION|COTTAGE|COUNTRY|GAINESVILLE|REVENUE|PROPERTIES|INDUSTRIAL|COPORATION|INTEREST|CITY|COMMUNICATIONS|COMPAN|COUNTY|DIVISION|HOSPITAL|L.L.C.|SUPPLY|GROUP|TECHNOLOGIES|ASSOCIATES|LIMITED|INSURANCE|L.P|JOINT|DESIGN|SRVCS|BANK|NORTH|INVEST|LTD|CENTER|TRUST|POPULAR|CLINIC|CREDIT|COMPANY|TELEPHONE|D.M.D.|UNIT|LLC|INDUSTRIES|C/O|CORPORATION|FINANCIAL|CREDIT|BANK|EMPLOYERS|FEDERAL|JEWELRY|AVENUE|ELECTRICAL|DISTRUBUTORS|UNIVERSITY|INNOVATIONS|SERVICE|SERVICES|CORP|CREATION|CERTIFIED|REPAIR|AUTO|TRANPORTATION|&|CO.|STAFFING|FIRM|TRADE|ENTERTAINMENT|TECHNOLOGY|FINANCE|CLEANING|SOLUTIONS|CASINO|HOLDINGS|HEALTH|SYSTEMS|L L C|FOODS|OIL|CLUB|UNIFUND|OFFICE| CO |CHANGE|PROJECT|ZOOLOGICAL|SPORTS|SHOP|BODY|BOATS|ABOUT|WIRELESS|WIRE|BEGINNING")
df['debtors_company_name'] = np.where(df['is_debtor_company']==True,df['debtors_full_name'],'')
df['debtors_last_name'] = np.where(df['is_debtor_company']==False,df['debtors_full_name'].str.split(',').str[0],'')
df['debtors_first_name'] = np.where(df['is_debtor_company']==False,
                                    df['debtors_full_name'].str.split(',').str[-1].str.strip().str.rsplit(' ').str[0],
                                    '')
df['debtors_middle_name'] = np.where(df['is_debtor_company']==False,
                                    df['debtors_full_name'].str.split(',').str[-1].str.strip().str.rsplit(' ').str[1],
                                    '')
# df['debtors_full_name'] = df['debtors_first_name']&','&df['debtors_middle_name']&','&df['debtors_last_name']



In [19]:
df['debtors_document_number'] = df['debtors_document_number'].str.split(':').str[-1].str.strip()
df['debtors_FEI_EIN_number'] = df['debtors_FEI_EIN_number'].str.split(':').str[-1].str.strip()

In [20]:
df['debtors_address'] = df['debtors_address_array'].str[0:-1].apply(lambda x: str(x).replace('[','').replace(']','').replace('\'', '')).str.replace(',',' ').str.strip().str.replace('|',',')
df['debtors_city'] = df['debtors_address_array'].str[-1].str.split('|').str[0]
df['debtors_state'] = df['debtors_address_array'].str[-1].str.split('|').str[-1].str.strip().str.split(' ').str[0]
df['debtors_zip'] = np.where(
                        df['debtors_address_array'].str[-1].str.split('|').str[-1].str.strip().str.split(' ').str[-1].str.isnumeric()
                        ,df['debtors_address_array'].str[-1].str.split('|').str[-1].str.strip().str.split(' ').str[-1],
                        df['debtors_address_array'].str[-1].str.split('|').str[-1].str.strip().str.split(' ').str[-2])

df['debtors_country'] = np.where(
                        df['debtors_address_array'].str[-1].str.split('|').str[-1].str.strip().str.split(' ').str[-1].str.isnumeric()
                        ,'',
                        df['debtors_address_array'].str[-1].str.split('|').str[-1].str.strip().str.split(' ').str[2])

df['debtors_appartment_number'] = np.where(
                    df['debtors_address_array'].str[0].str.contains('SUITE|APT'),
                    df['debtors_address_array'].str[0].str.split('SUITE').str[-1].str.split('APT.').str[-1].str.split(' APT').str[-1],
                    ''
    )


In [21]:
df = df.drop(columns=['debtors_arrays','debtors_arrays_count','debtors_line_number_FEI_EIN','debtors_line_number_document','debtors_address_last_line_number','is_debtor_company',
                    'debtors','debtors_address_array','creditor_first_line','creditor_last_line','is_creditor_company','creditor_name','creditors'
])

In [22]:
columns = [
    'doc_number', 'status', 'case_number', 'court_name', 'file_date', 'entry_date', 
    'expiry_date', 'amount_due', 'interest_rate', 'original_doc', 'amount_remaining',
    'creditor_company_name', 'creditor_full_name', 'creditor_fname', 'creditor_mname', 
    'creditor_lname', 'creditor_address', 'creditor_city', 'creditor_zip', 'creditor_state',
    'debtors_company_name', 'debtors_full_name', 'debtors_first_name', 'debtors_middle_name', 
    'debtors_last_name', 'debtors_address', 'debtors_appartment_number', 'debtors_city', 
    'debtors_state', 'debtors_zip', 'debtors_country', 'debtors_document_number', 
    'debtors_FEI_EIN_number', 'url'
]
df = df[columns]

In [23]:
with pd.ExcelWriter('sunbiz.xlsx', engine='xlsxwriter') as writer:
    df.iloc[0:400000].to_excel(writer, sheet_name=f'sunbiz_1', index=False)
    df.iloc[400001:800000].to_excel(writer, sheet_name=f'sunbiz_2', index=False)
    df.iloc[800001:-1].to_excel(writer, sheet_name=f'sunbiz_3', index=False)

##### FINAL RESULT WILL SAVED AS 'subiz.xlsx'